# Importing Libraries we will need.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
import glob
from PIL import Image
from numpy import expand_dims,zeros,ones
from numpy.random import randn,randint
from sklearn.model_selection import train_test_split
from tensorflow.keras.datasets.mnist import load_data
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Reshape, Flatten, Conv2D, Conv2DTranspose, LeakyReLU, ReLU
from tensorflow.keras.layers import Dropout, Embedding, Concatenate, BatchNormalization
from tensorflow.keras.initializers import RandomNormal

# Setting up basic parameters

In [ ]:
IMAGE_SIZE = (28,28,1)
BATCH_SIZE = 200
STEPS = 500
EPOCHS = 20
LATENT_DIM = 100
N_CLASSES = 10

# Fixed Values

Here we are fixing z_vis which is basically a noise vector. At every epoch we will pass this to the generator to see how the generator is improving for the same set of noise. 

y_vis is basically a one hot encoded array of out labels (0-9). We will use to see how effective the model is for generating the images of the different labels. 

In [ ]:
z_vis = tf.random.normal([10, LATENT_DIM])
y_vis = tf.constant(np.eye(10), dtype='float32')

# Loading the Dataset

We will load the data from the csv and then reshape it into how the images were. 

In [ ]:
train_df = pd.read_csv("../input/digit-recognizer/train.csv")
y = train_df['label']
train_df.drop('label',axis=1,inplace=True)
training_images=train_df.to_numpy()
training_images = training_images.reshape(42000, 28, 28, 1)

We will be splitting the data into train and test sets. We will then be normalizing our training data for better performance. We will also one hot encode the labels which will improve our training process.

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(training_images,y,test_size=0.2)
x_train = x_train / 255.0
y_train = tf.one_hot(y_train, depth=10, dtype='float32')
data_iter = iter(tf.data.Dataset.from_tensor_slices((x_train, y_train)).shuffle(4 * BATCH_SIZE).batch(BATCH_SIZE).repeat())

# Visualize the Data

In [ ]:
for i in range(25):
    plt.subplot(5, 5, 1 + i)
    plt.axis('off')
    plt.imshow(x_train[i], cmap='gray_r')
plt.show()

# Generator

In a GAN the role of a generator is to  take noise as input and create as realistic image as possible. In a conditional gan we also provide the label alongside the noise to tell the model what type of image it should try to generate.

In [ ]:
def Generator():
    z = Input(shape=(LATENT_DIM,), dtype='float32')#input for noize vector
    y = Input(shape=(10,), dtype='float32')#input for the label

    x = Concatenate()([z, y])
    x = Dense(7 * 7 * 128)(x)
    x = Reshape((7, 7, 128))(x)

    x = Conv2DTranspose(128, 5, 2, 'same')(x)
    x = BatchNormalization()(x)
    x = LeakyReLU(0.2)(x)
    
    x = Conv2DTranspose(64, 5, 2, 'same')(x)
    x = BatchNormalization()(x)
    x = LeakyReLU(0.2)(x)

    out = Conv2DTranspose(1,  5, 1, 'same', activation='sigmoid')(x)

    return tf.keras.Model(inputs=[z, y], outputs=out)

# Discriminator

In GANs the role of the discriminator is to differentiate between the real and fake images.

In [ ]:
def Discriminator():
    X = Input(shape=(28, 28), dtype='float32')#input for real or fake images
    Y = Input(shape=(10,), dtype='float32')#input for the label

    y = tf.tile(tf.reshape(Y,[-1, 1, 1, 10]), [1, 28, 28, 1])
    x = Reshape((28, 28, 1))(X)
    x = Concatenate()([x, y])

    x = Conv2D(32,  5, 2, 'same')(x)

    x = Conv2D(64,  4, 2, 'same')(x)
    x = BatchNormalization()(x)
    x = LeakyReLU(0.2)(x)
    
    x = Conv2D(128,  3, 2, 'same')(x)
    x = BatchNormalization()(x)
    x = LeakyReLU(0.2)(x)
    
    x = Flatten()(x)
    x = Dense(128)(x)
    x = BatchNormalization()(x)

    out = Dense(1)(x)

    return tf.keras.Model(inputs=[X, Y], outputs=out)

In [ ]:
G = Generator()
D = Discriminator()

# Loss Functions

For the generator the goal is to make as realisitc images as possible. For it the loss is high when discriminates labels its generated images as fake. 

For the discriminator the goal is to label the fake images as fake and real as real. For it loss is high when the generator creates realisitic images and fools the the discriminator.

In [ ]:
cross_entropy = tf.keras.losses.BinaryCrossentropy(from_logits = True)
def G_loss(D, x_fake, y):
    return cross_entropy(tf.ones_like(D([x_fake, y])), D([x_fake, y]))
def D_loss(D, x_real, x_fake, y):
    return cross_entropy(tf.ones_like(D([x_real, y])), D([x_real, y])) + cross_entropy(tf.zeros_like(D([x_fake, y])), D([x_fake, y]))

In [ ]:
G_opt = Adam(2e-4,0.5)
D_opt = Adam(2e-4,0.5)

# Function to save images at specific epochs

In [ ]:
def generate_and_save_images(model, epoch, noise,label):
    predictions = model([noise,label])

    fig = plt.figure(figsize=(8, 2))

    for i in range(predictions.shape[0]):
        plt.subplot(2, 5, i+1)
        plt.imshow(model([noise,label])[i,:,:] * 255.0)
        plt.axis('off')
    plt.tight_layout()
    plt.savefig('image_at_epoch_{:04d}.png'.format(epoch))
    plt.show()

# Custom Training Loop

reference: https://www.tensorflow.org/tutorials/generative/dcgan

In [ ]:
for epoch in range(EPOCHS):
    for step in range(STEPS):
        z_mb = tf.random.normal([BATCH_SIZE, LATENT_DIM])
        x_real, y = next(data_iter)
        with tf.GradientTape() as G_tape, tf.GradientTape() as D_tape:  
            x_fake = G([z_mb, y])
            G_loss_curr = G_loss(D, x_fake, y)
            D_loss_curr = D_loss(D, x_real, x_fake, y)

        G_grad = G_tape.gradient(G_loss_curr, G.trainable_variables)
        D_grad = D_tape.gradient(D_loss_curr, D.trainable_variables)

        G_opt.apply_gradients(zip(G_grad, G.trainable_variables))
        D_opt.apply_gradients(zip(D_grad, D.trainable_variables))
    
    print('epoch: {}; G_loss: {:.6f}; D_loss: {:.6f}'.format(epoch+1, G_loss_curr, D_loss_curr))
    generate_and_save_images(G,epoch,z_vis,y_vis)

# Evaluation

Here we will check the performance of our model by seeing how well it is performing in generating images of each label

In [ ]:
new_noise = tf.random.normal([100, LATENT_DIM])
labels = np.asarray([np.eye(10)[i//10] for i in range(100)])
pred = G([new_noise,labels])
for i in range(100):
    plt.subplot(10, 10, 1 + i)
    plt.axis('off')
    plt.imshow(pred[i, :, :], cmap='gray_r')
plt.show()

# Save the images generated during training as a gif

In [ ]:
fp_in = "./image_*.png"
fp_out = "./MNIST_training.gif"

img, *imgs = [Image.open(f) for f in sorted(glob.glob(fp_in))]
img.save(fp=fp_out, format='GIF', append_images=imgs,
         save_all=True, duration=200, loop=0)

# Save the models for future use

In [ ]:
G.save('mnist_gen.hdf5')
D.save('mnist_dis.hdf5')

If you found this notebook helpful please dont forget to leave an upvote!!!